In [ ]:
import torch
from torch.utils import data
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import random
import numpy as np
import math

from evaluation_utils import create_model

import sys
sys.path.append("../")
from plot_utils import plot_points, plot_points_in_multiple_subplots, plot_images_in_grid
from dataset_utils import ImageNet, Places365, ArtPlaces

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print(torch.cuda.get_device_name())

### Konstanten

In [ ]:
IMAGE_NUM = 1000
BATCH_SIZE = 128
FEATURE_EXTRACTION_METHOD = "t-sne"
PLOT_EACH_MODEL = False

MODELS = [
    # {
    #     "name": "Perceptual Loss",
    #     "architecture": "ResNet18PerceptualLoss",
    #     "dataset": ["ImageNet"] # ["ImageNet", "Places365", "ArtPlaces"]
    # },
    # {
    #     "name": "Siamese Network",
    #     "architecture": "ResNet18SiameseNetwork",
    #     "weights": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\siamese_network\resnet18_imagenet_20250111_161159\state_dict_49.pt",
    #     "dim": 1000,
    #     "dataset": ["ImageNet"] # ["ImageNet", "Places365", "ArtPlaces"]
    # },
    {
        "name": "MoCo",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": "MOCO",
        "weights_trained": None,
        "dim": 128,
        "K": 65536,
        "mlp": False,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on Places365)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_places365_20251125_155309\state_dict_50.pt",
        "dim": 128,
        "K": 65536,
        "mlp": True,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on Places365)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_places365_20251125_111703\state_dict_50.pt",
        "dim": 128,
        "K": 65536,
        "mlp": True,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on Places365)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_places365_20251125_002737\state_dict_150.pt",
        "dim": 128,
        "K": 65536,
        "mlp": True,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on Places365)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_places365_20251124_172431\state_dict_45.pt",
        "dim": 128,
        "K": 65536,
        "mlp": True,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on Places365)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_places365_20251124_140015\state_dict_40.pt",
        "dim": 128,
        "K": 65536,
        "mlp": False,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on ArtPlaces)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_artplacestimesn_20251113_170955\state_dict_40.pt",
        "dim": 128,
        "K": 4544,
        "mlp": False,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on ArtPlaces)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_artplacestimesn_20251113_100305\state_dict_120.pt",
        "dim": 128,
        "K": 4544,
        "mlp": False,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on ArtPlaces)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_artplacestimesn_20251113_085121\state_dict_20.pt",
        "dim": 128,
        "K": 4544,
        "mlp": False,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    {
        "name": "MoCo (trained on ArtPlaces)",
        "architecture": "MoCo",
        "model": "resnet50",
        "weights_pretrained": None,
        "weights_trained": r"C:\Users\mariu\Documents\Studium\Praktikum\Gewichte\moco\resnet50_artplacestimesn_20251113_005000\state_dict_120.pt",
        "dim": 128,
        "K": 4544,
        "mlp": False,
        "dataset": ["ImageNet", "Places365", "ArtPlaces"] # ["ImageNet", "Places365", "ArtPlaces"]
    },
    # {
    #     "name": "DINOv2",
    #     "architecture": "DINO_v2",
    #     "model": "dinov2_vitb14",
    #     "dataset": ["ImageNet"] # ["ImageNet", "Places365", "ArtPlaces"]
    # },
    # {
    #     "name": "CLIP",
    #     "architecture": "CLIP",
    #     "model": "ViT-B/32",
    #     "dataset": ["ImageNet"] # ["ImageNet", "Places365", "ArtPlaces"]
    # },
]

CLASSES_IMAGENET = [
    "tabby",
    "tiger cat",
    "Persian cat",
    "Egyptian cat",
    "cougar",
    "lynx",
    "German shepherd",
    "French bulldog",
    "Eskimo dog",
    "brown bear",
    "tractor",
    "warplane",
    "passenger car"
    
]
# CLASSES_IMAGENET = [
#     "tabby",
#     "tiger cat",
#     # "Persian cat",
#     # "Egyptian cat",
#     "cougar",
#     "lynx",
#     # "polecat",
#     # "tiger",
#     "tractor",
#     # "goldfinch",
#     "warplane",
#     # "garbage truck",
#     # "grand piano",
#     "German shepherd",
#     "French bulldog",
#     "Eskimo dog",
#     # "Siamese cat",
#     # "cougar",
#     # "Madagascar cat",
#     # "lesser panda",
#     "brown bear",
#     # "lion",
#     "passenger car"
# ]
COLORS_IMAGENET = None
# COLORS_IMAGENET = [
#     "green",
#     "green",
#     "green",
#     "green",
#     "lightgreen",
#     "lightgreen",
#     "lightblue",
#     "lightgreen",
#     "red",
#     "lightblue",
#     "red",
#     "red",
#     "red",
#     "lightblue",
#     "lightblue",
#     "lightblue",
#     "magenta",
#     "orchid",
#     "orchid",
#     "darkviolet",
#     "darkviolet",
#     "orchid",
#     "indigo"
# ]
CLASSES_PLACES365 = [
    "amphitheater",
    "castle",
    "palace",
    "aqueduct",
    "alley",
    "highway",
    "beer_hall",
    "chalet",
    "botanical_garden",
    "beach",
    "mountain",
    "canyon",
]
# CLASSES_PLACES365 = [
#     "church/indoor",
#     "church/outdoor",
#     "synagogue/outdoor",
#     "mosque/outdoor",
#     "forest/broadleaf"
# ]
# CLASSES_PLACES365 = [
#     "church/outdoor",
#     "palace",
#     "castle",
#     "forest/broadleaf",
#     "beach"
# ]
COLORS_PLACES365 = None
CLASSES_ARTPLACES = [
    "amphitheater",
    "castle",
    "palace",
    "aqueduct",
    "alley",
    "highway",
    "beer_hall",
    "chalet",
    "botanical_garden",
    "beach",
    "mountain",
    "canyon",
]
COLORS_ARTPLACES = None

### Datensatz & Teildatensatz

In [ ]:
imagenet = ImageNet(root=r"C:\Users\mariu\Documents\Development\Datasets\imagenet-object-localization-challenge")
imagenet_subset = imagenet.getSubset(IMAGE_NUM, CLASSES_IMAGENET)
imagenet_dataloader = data.DataLoader(imagenet_subset, batch_size=BATCH_SIZE)

places365 = Places365(root=r"C:\Users\mariu\Documents\Studium\Praktikum\Places365_Evaluation_Subset")
places365_subset = places365.getSubset(IMAGE_NUM, CLASSES_PLACES365)
places365_dataloader = data.DataLoader(places365_subset, batch_size=BATCH_SIZE)

artplaces = ArtPlaces(root=r"C:\Users\mariu\Documents\Development\Datasets\ArtPlaces_13371280")
artplaces_subset = artplaces.getSubset(IMAGE_NUM, CLASSES_ARTPLACES)
artplaces_dataloader = data.DataLoader(artplaces_subset, batch_size=BATCH_SIZE)

datasets = {
    "ImageNet": imagenet,
    "ImageNet_Loader": imagenet_dataloader,
    "ImageNet_Classes": CLASSES_IMAGENET,
    "ImageNet_Colors": np.r_[plt.colormaps["rainbow"](np.linspace(0, 1, len(CLASSES_IMAGENET)))[::-1], [[0.95, 0.95, 0.95, 1]]].tolist() if COLORS_IMAGENET is None else COLORS_IMAGENET,
    "Places365": places365,
    "Places365_Loader": places365_dataloader,
    "Places365_Classes": CLASSES_PLACES365,
    "Places365_Colors": np.r_[plt.colormaps["rainbow"](np.linspace(0, 1, len(CLASSES_PLACES365)))[::-1], [[0.95, 0.95, 0.95, 1]]].tolist() if COLORS_PLACES365 is None else COLORS_PLACES365,
    "ArtPlaces": artplaces,
    "ArtPlaces_Loader": artplaces_dataloader,
    "ArtPlaces_Classes": CLASSES_ARTPLACES,
    "ArtPlaces_Colors": np.r_[plt.colormaps["rainbow"](np.linspace(0, 1, len(CLASSES_ARTPLACES)))[::-1], [[0.95, 0.95, 0.95, 1]]].tolist() if COLORS_ARTPLACES is None else COLORS_ARTPLACES
}

### Architektur

In [ ]:
def extract_features(vectors, n_components=2):
    vectors_numpy = torch.flatten(vectors, start_dim=1).numpy()

    match FEATURE_EXTRACTION_METHOD:
        case "pca":
            fe = PCA(n_components=n_components)
            fe.fit(vectors_numpy)
            features = fe.transform(vectors_numpy)
        case "t-sne":
            fe = TSNE(n_components=n_components)
            features = fe.fit_transform(vectors_numpy)
    return features

In [ ]:
def compute_vecotrs(model, dataset, dataloader, classes, colors):
    images, labels = [],[]
    for i, l in dataloader:
        images.append(model(torch.clone(i).to(device)).cpu())
        labels.append(l)
    images = torch.cat(images)
    labels = torch.cat(labels)
    features_numpy = extract_features(images)
    labels_numpy = labels.numpy()

    legend = {}
    color_dict = {}
    for i in torch.unique(labels):
        legend[i.item()] = dataset.idx_to_class[i.item()]
        if colors is not None:
            color_dict[i.item()] = colors[classes.index(legend[i.item()])]
        else:
            color_dict = None
    
    return features_numpy, labels_numpy, legend, color_dict

In [ ]:
%matplotlib qt

features_numpy_list = []
labels_numpy_list = []
legend_list = []
color_dict_list = []
title_list = []

for model_info in MODELS:
    print(f"Processing model: {model_info['name']}")
    model = create_model(model_info, device=device)
    for dataset in model_info["dataset"]:
        features_numpy, labels_numpy, legend, color_dict = compute_vecotrs(model, datasets[dataset], datasets[f"{dataset}_Loader"], datasets[f"{dataset}_Classes"], datasets[f"{dataset}_Colors"])

        # title = f"{model_info['name']} - {dataset}"
        if PLOT_EACH_MODEL:
            plot_points(features_numpy, labels_numpy, legend=legend, title=None, colors=color_dict, figsize=(15, 10), label_cols=4, fontsize=25)

        features_numpy_list.append(features_numpy)
        labels_numpy_list.append(labels_numpy)
        legend_list.append(legend)
        color_dict_list.append(color_dict)
        title_list.append(f"{model_info['name']} - {dataset}")

In [ ]:
cols = math.ceil(math.sqrt(len(features_numpy_list)))
rows = math.ceil(len(features_numpy_list) / cols)

In [ ]:
plot_points_in_multiple_subplots(
    points=features_numpy_list,
    labels=labels_numpy_list,
    legend=legend_list,
    subplot_titles=title_list,
    nrows=rows,
    ncols=cols,
    colors=color_dict_list,
    label_cols=4,
    fontsize=10
)

In [ ]:
plot_points_in_multiple_subplots(
    points=[f[0:len(f):2] for f in features_numpy_list],
    labels=[l[0:len(l):2] for l in labels_numpy_list],
    legend=legend_list,
    subplot_titles=title_list,
    nrows=rows,
    ncols=cols,
    colors=color_dict_list,
    label_cols=4,
    fontsize=10
)